In [1]:
import numpy as np
import pandas
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

In [14]:
emotions = ['happy', 'sad', 'angry', 'chill', 'dreamy', 'lovey-dovey', 'epic', 'anxious', 'pump-up', 'sexy']

client_id = '2d0aef4304064e67bdce269495c514ba'
client_secret = 'b0b2829b32cd4963a5de4c35528faf1d'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

songs = []

for e in emotions:

    search_query = e
    result = sp.search(search_query, type='playlist')
    print(e + '\n')

    for i in result['playlists']['items']:

        playlist_id = i['id']
        tracks = sp.playlist_items(playlist_id)

        for j in tracks['items']:
            if (j['track'] != None):
                track_id = j['track']['id']
                if (track_id != None and track_id != ''):
                    #details
                    uri = 'spotify:track:' + track_id
                    features = sp.audio_features(uri)
                    if (features[0] != None):
                        songs.append([e, track_id, features[0]['danceability'], features[0]['energy'], features[0]['key'], features[0]['loudness'], features[0]['mode'], features[0]['speechiness'], features[0]['acousticness'], features[0]['instrumentalness'], features[0]['valence'], features[0]['tempo']])

happy

sad

angry

chill

dreamy

lovey-dovey

epic

anxious

pump-up

sexy



In [26]:
array = np.array(songs)
df = pandas.DataFrame(array, columns=['emotion', 'song', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'valence', 'tempo'])
df.describe()

,emotion,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,valence,tempo
count,8417,8417,8417,8417,8417,8417,8417,8417,8417,8417,8417,8417
unique,10,7575,848,1113,12,5767,2,1098,2415,2528,1238,6482
top,pump-up,7lPN2DXiMsVn7XUKtOW1CS,0.671,0.473,0,-8.761,1,0.0382,0.952,0,0.12,99.991
freq,960,8,39,25,990,10,5307,34,21,3210,28,9


In [34]:
df.drop_duplicates(subset='song', inplace=True)
df.reset_index(drop=True, inplace=True)
df.describe()

,emotion,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,valence,tempo
count,7575,7575,7575,7575,7575,7575,7575,7575,7575,7575,7575,7575
unique,10,7575,848,1113,12,5767,2,1098,2415,2528,1238,6482
top,dreamy,7zgqtptZvhf8GEmdsM2vp2,0.671,0.537,0,-5.122,1,0.0335,0.128,0,0.503,120.002
freq,886,1,29,22,891,7,4755,28,15,2746,21,8


In [36]:
df.head(len(df))

,emotion,song,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,valence,tempo
0,happy,1FHZyJ35kZ8DaLVOlhOIYK,0.523,0.779,4,-3.462,1,0.0654,0.00484,4.05e-06,0.465,110.041
1,happy,3uxsrlRldPneshvxlXVcYS,0.754,0.792,1,-5.278,0,0.0429,0.0071,3.58e-06,0.791,118.024
2,happy,5qpEtUsEI1VkG63MNlRrAC,0.598,0.559,5,-5.835,1,0.0387,0.019,0,0.19,122.066
3,happy,1LLm4XXXV0FErcxPDw9Vt7,0.679,0.73,4,-4.727,1,0.0498,0.0967,7.44e-06,0.587,123.96
4,happy,7pDdE5FLqcNCvjxzRzm80r,0.753,0.749,6,-5.214,1,0.0314,0.0546,4.39e-06,0.589,121.988
...,...,...,...,...,...,...,...,...,...,...,...,...
7570,sexy,4gKCKkQbtejYebL9Km5cu1,0.636,0.418,4,-10.536,1,0.0316,0.503,0,0.347,96.132
7571,sexy,2Bm2pjLh2wlYwZ4nrcY6wU,0.663,0.149,4,-14.239,0,0.0466,0.644,2.5e-05,0.397,107.19
7572,sexy,5wXQFi7alwkKVf7PdYb3Co,0.536,0.533,11,-7.54,1,0.0474,0.354,0.000275,0.139,78.019
7573,sexy,7czq3qsGLwbedHtuQYUXOc,0.765,0.44,7,-8.71,0,0.108,0.776,2.33e-05,0.52,80.013
